# Proper Orthogonal Decomposition for Reduced Order Modelling 

<a id="table"></a>

## Table of contents
- [Introduction](#intro)
- [High resolution model](#HR)
- [Proper orthogonal decomposition](#POD)
- [Low resolution model](#LR)
- [Performance improvement](#improvement)
- [Error between high and low resolution models](#error)

In [17]:
from time import process_time
import numpy as np
from scipy import linalg
from scipy import sparse

from modules.rom import pod
from scripts import animation
# reservoir model
from scripts import water_canals_model as model
from modules.simulator import simplots
from modules.simulator.units import UnitRegistry

# Units
u = UnitRegistry()

# Set image folder path
imgpath = ".\\images"

<a id="intro"></a>

## [Introduction](#table)

 Reduced order modeling (ROM) has the aim of reducing the number of dimensions (equations) that represent a model in order to solve it faster. In this notebook I will show an example of how to do ROM with proper orthogonal decomposition (POD). A good explanation of POD can be found in this [document](#link).

<a id="HR"></a>

## [High resolution model (HRM)](#table)

For this example the model is a single layer of rock of low permeability with two canals of high permeability. There is a single phase flow of water caused by a difference of pressure between the left face and right face of the rock. [This image](#perm) shows the permeability distribution.

The model has a grid with 10,000 cells and it is solved with the Lagging Coefficients method. So for each timestep a linear system with dimensions 10,000 x 10,000 is solved.

The simulation is run fur 200 timesteps equivalent to 2 days. Solving the HRM takes approximately **175 seconds** to run.

In [2]:
model.grid.cellnumber

5000

In [ ]:
# Plot permeability
log_perm = np.log(model.rock.perm/u.darcy / u.milli)
simplots.plotCellValues2D(model.grid, log_perm, 'inferno', np.min(log_perm), np.max(log_perm),
                     title='Permeability Ln(mD)', 
                     filename='{}\\pod_perm'.format(imgpath))

<a id="perm">
        <img src=".\images\pod_perm.png?" style="width:400px;height:-1px;" >
</a> 

In [18]:
t = process_time()
# Run simulation
X, well_solution, sch = model.LCM.solve(model.sch, max_inner_iter = 1, tol = 1E-6, ATS = False)
HRM_time = process_time() - t
model.nice_print('Time running simulation = {:.3f} seconds.'.format(HRM_time))

 Time running simulation = 50.404 seconds.

In [ ]:
# Transform units to psi
p = X/u.psi
# We will  save a plot for each timestep to create an animation
acum_sum = np.cumsum(model.sch.timesteps) / u.day
# Add "0" for the initial conditions
acum_sum = np.hstack((0, acum_sum))

for k in np.arange(0,acum_sum.size, 1): 
    model.plotCellValues2D(model.grid, p[:, k], 'inferno', np.min(p), np.max(p),
                     title='Pressure (psi). {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\pod_pw{}'.format(imgpath, k))

In [ ]:
#Make gif
animation.make_gif(imgpath, "pod_pw")

In [ ]:
# Delete intermediary images
animation.delete_files(imgpath,"pod_pw","png")

The following animation shows the pressure distribution in the rock layer. As it is expected, the pressure increases  mainly in the high permeability area of the canals.

<img src=".\images\pod_pw.gif" >

<a id = POD></a>

## [Proper orthogonal decomposition (POD)](#table)

Here I implement the POD technique as explained in this [paper]() by Andrea. 

In [4]:
# Covariance matrix
C = X.T.dot(X)
# Singular value decomposition
V,si,_ = linalg.svd(C)
# Calculate basis
U = (X.dot(V)) * si ** (1/2)
print(U.shape)

(5000, 201)


The matrix $U$ contains the basis vectors. To form the reduced basis matrix $\:\phi$, only the basis vectors that contribute the most energy are selected. The following plots show that the first vector captures almost all of the energy. Also, there is a change of slope at vector 15. 

In [ ]:
pod.plot_energy(si, "{}\\pod_energy".format(imgpath))
pod.plot_energy(si[:40], "{}\\pod_energy_closeup".format(imgpath))

**Energy of the basis vectors**
<img src = ".\images\pod_energy.png?" style="width:400px;height:-1px;" >
**Close up**
<img src = ".\images\pod_energy_closeup.png?" style="width:400px;height:-1px;" >

In [5]:
basis_size = 15
reduced_basis = U[:,:basis_size]

<a id="LR"></a>

## [Low resolution model (LRM)](#table)

In [6]:
linear_solver = lambda A, b : pod.linear_solver(A, b, sparse.csr_matrix(reduced_basis))

In [19]:
t = process_time()
# Run simulation
Xr, well_solution, sch = model.LCM.solve(model.sch, max_inner_iter = 1, tol = 1E-6, ATS = False, linear_solver = linear_solver)
LRM_time = process_time() - t
model.nice_print('Time running simulation = {:.3f} seconds.'.format(LRM_time))

 Time running simulation = 54.944 seconds.

In [8]:
# Transform units to psi
p = Xr/u.psi
# We will  save a plot for each timestep to create an animation
acum_sum = np.cumsum(model.sch.timesteps) / u.day
# Add "0" for the initial conditions
acum_sum = np.hstack((0, acum_sum))

for k in np.arange(0,acum_sum.size, 1): 
    model.plotCellValues2D(model.grid, p[:, k], 'inferno', np.min(p), np.max(p),
                     title='Pressure (psi). {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\pod_LRM_pw{}'.format(imgpath, k))

In [9]:
#Make gif
animation.make_gif(imgpath, "pod_LRM_pw")

In [10]:
# Delete intermediary images
animation.delete_files(imgpath,"pod_LRM_pw","png")

The solution appying the POD technique.

<img src=".\images\pod_LRM_pw.gif" >

<a id="improvement"></a>

## [Performance improvement](#table)

The total simulation time is reduced by  almost 19%.

In [20]:
improvement = (HRM_time - LRM_time )/ HRM_time 
print('Save {:.2f}% of the time.'.format(improvement * 100))

Save -9.01% of the time.


<a id="error"></a>

## [Error between high and low resolution models](#table)

Depending of the basis the is selected for the projection the quality of the solution changes. The bigger the number of 

The next animation shows the percentage difference between the high resolution and low resolution solutions for each cell for each timestep. The maximum difference between solutions is 12 %. 

In [12]:
# X is the solution from the HRM and Xr is the solution from the LRM
#diff =np.subtract(a,b)
diff = np.subtract(X/u.psi,Xr/u.psi)
diff = np.divide(diff,X/u.psi) * 100

#np.set_printoptions(precision=3, suppress = True)

#print (np.linalg.norm(diff, ord=np.inf))
max_diff = (np.max(np.abs(diff)))
print('Maximum difference between solutions: {:.2f}%.'.format(max_diff))
#col_diff = np.linalg.norm(diff, ord = np.inf, axis =0)
#print(col_diff)

Maximum difference between solutions: 13.60%.


In [13]:
diff = np.abs(diff)
for k in np.arange(0,acum_sum.size, 1): 
    model.plotCellValues2D(model.grid, diff[:, k], 'inferno', np.min(diff), np.max(diff),
                     title='Relative error (%). {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\pod_error{}'.format(imgpath, k))

In [14]:
#Make gif
animation.make_gif(imgpath, "pod_error")

In [15]:
# Delete intermediary images
animation.delete_files(imgpath,"pod_error","png")

<img src = ".\images\pod_error.gif" style="width:400px;height:-1px;" >

## [To the top](#table)

In [16]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Using the style sheet found here  Lorena Barba /* https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css */